## 1. 数据库风格的DataFrame合并
`merge()` 通过列/行索引将两个DataFrame合并起来  
参数  
- `how`：连接方式，可用值{'left', 'right', 'outer', 'inner'}，默认'inner'，各值说明如下：![title](img/marge-how.png)
- `on`：指定使用哪列/行作为连接键（必须是共同拥有的标签，或标签组成的list），如果没有指定，会默认使用所有共同拥有的标签的行/列进行连接
- `left_on` `right_on`：指定左/右对象用做连接键的列
- `left_index` `right_index`：布尔值，是否使用左/右对象的index索引用做连接键
- `sort`：布尔值，合并后的数据根据连接键进行排序，默认False
- `suffixes`：字符串值元组，用于追加到重叠标签名的末尾，以区分数据来源，默认`('_x','_y')`
- `indicator`：布尔值，在合并结果中增加一列'_merge'，用来指明每行数据的来源，默认False

In [1]:
import numpy as np
import pandas as pd

In [2]:
d1 = pd.DataFrame({'key1': list('abcabcbac'), 'data1': np.random.randint(
    10, size=9), 'k': list('ZZXYYXZYX')})
d1

,key1,data1,k
0,a,8,Z
1,b,9,Z
2,c,6,X
3,a,2,Y
4,b,5,Y
5,c,1,X
6,b,3,Z
7,a,5,Y
8,c,4,X


In [3]:
d2 = pd.DataFrame({'key2': list('abd'), 'k': list('XYZ')})
d2

,key2,k
0,a,X
1,b,Y
2,d,Z


In [4]:
pd.merge(d1, d2, on='k')  # 使用共同的k列作为连接键

,key1,data1,k,key2
0,a,8,Z,d
1,b,9,Z,d
2,b,3,Z,d
3,c,6,X,a
4,c,1,X,a
5,c,4,X,a
6,a,2,Y,b
7,b,5,Y,b
8,a,5,Y,b


In [5]:
# 分别指定两个表不同名的列作为连接键，默认how='inner'交集，显示两个表连接键共有的值
pd.merge(d1, d2, left_on='key1', right_on='key2')

,key1,data1,k_x,key2,k_y
0,a,8,Z,a,X
1,a,2,Y,a,X
2,a,5,Y,a,X
3,b,9,Z,b,Y
4,b,5,Y,b,Y
5,b,3,Z,b,Y


In [6]:
# how='outer'并集，显示两个表连接键的所有的值
pd.merge(d1, d2, left_on='key1', right_on='key2', how='outer')

,key1,data1,k_x,key2,k_y
0,a,8.0,Z,a,X
1,a,2.0,Y,a,X
2,a,5.0,Y,a,X
3,b,9.0,Z,b,Y
4,b,5.0,Y,b,Y
5,b,3.0,Z,b,Y
6,c,6.0,X,NaN,NaN
7,c,1.0,X,NaN,NaN
8,c,4.0,X,NaN,NaN
9,NaN,NaN,NaN,d,Z


In [7]:
# how='left'，显示左表连接键的所有的值
# sort=True，根据连接键进行排序
# suffixes=('_L','_R')，两个表相同的列名增加后缀用以区分
# indicator=True，增加'_merge'列，用来显示所在行的数据来源
pd.merge(d1, d2, left_on='key1', right_on='key2', how='left',
         sort=True, suffixes=('_L', '_R'), indicator=True)

,key1,data1,k_L,key2,k_R,_merge
0,a,8,Z,a,X,both
1,a,2,Y,a,X,both
2,a,5,Y,a,X,both
3,b,9,Z,b,Y,both
4,b,5,Y,b,Y,both
5,b,3,Z,b,Y,both
6,c,6,X,NaN,NaN,left_only
7,c,1,X,NaN,NaN,left_only
8,c,4,X,NaN,NaN,left_only


## 索引上的合并
对于连接键位于索引上时候，可以指定`left_index=True`或`right_index=True`，用以说明索引作为连接键

In [8]:
# 将d2的k列作为index，并保留k列且修改k列的值
d2.set_index('k',inplace=True)
d2

,key2
k,
X,a
Y,b
Z,d


In [9]:
# d2使用索引作为连接键和d1的k列进行连接
pd.merge(d1,d2,left_on='k',right_index=True)

,key1,data1,k,key2
0,a,8,Z,d
1,b,9,Z,d
6,b,3,Z,d
2,c,6,X,a
5,c,1,X,a
8,c,4,X,a
3,a,2,Y,b
4,b,5,Y,b
7,a,5,Y,b


对于层次化的索引，必须将多个对应的列以列表形式作为连接键来使用

In [10]:
data1 = pd.DataFrame(
    np.random.randint(1, 100, size=25).reshape((5, 5)),
    index=[list('aabbc'), list('xyyzz')],
    columns=list('ABCDE'))
data1

A   B   C   D   E
a x  11  71  43  41  60
  y  78  88  73  89  34
b y  19  89  20  57  70
  z  98   1  26  65  36
c z  44  55  36  50  12

In [11]:
data2 = pd.DataFrame(
    {'key1': list('bbaac'), 'key2': list('xzxyz'), 'value': list('XZYZX')})
data2

,key1,key2,value
0,b,x,X
1,b,z,Z
2,a,x,Y
3,a,y,Z
4,c,z,X


In [12]:
# 将右表data2中的key1和key2组成列表作为连接键，与data1的多层index进行连接
pd.merge(data1,data2,left_index=True,right_on=['key1','key2'],how='outer')

,A,B,C,D,E,key1,key2,value
2,11.0,71.0,43.0,41.0,60.0,a,x,Y
3,78.0,88.0,73.0,89.0,34.0,a,y,Z
4,19.0,89.0,20.0,57.0,70.0,b,y,NaN
1,98.0,1.0,26.0,65.0,36.0,b,z,Z
4,44.0,55.0,36.0,50.0,12.0,c,z,X
0,NaN,NaN,NaN,NaN,NaN,b,x,X


### join()方法按索引合并
- `join()`方法可以实现按索引合并两个表，但要求没有重叠的列，并且默认使用左连接。
- 一个使用列作为连接键的表也可以使用`join()`方法来合并一个使用索引作为连接键的表，需要使用`on=`来指明作为连接键的列。
- `join()`方法还可以按索引同时合并多个表，需合并的表以列表形式传入，**注意：不支持列，所有表都必须按索引合并**

In [13]:
data3 = pd.DataFrame(
    np.random.randint(1, 100, size=5).reshape((5, 1)),
    index=[list('bbaac'), list('xzxyz')],columns=['value'])
data3

value
b x     82
  z      3
a x     75
  y     39
c z     83

In [14]:
data1.join(data3) # join按索引进行合并，默认使用左连接，左表所有索引都保留

A   B   C   D   E  value
a x  11  71  43  41  60   75.0
  y  78  88  73  89  34   39.0
b y  19  89  20  57  70    NaN
  z  98   1  26  65  36    3.0
c z  44  55  36  50  12   83.0

In [15]:
data1.join(data3,how='outer') # 可以使用how来修改连接方式

A     B     C     D     E  value
a x  11.0  71.0  43.0  41.0  60.0   75.0
  y  78.0  88.0  73.0  89.0  34.0   39.0
b x   NaN   NaN   NaN   NaN   NaN   82.0
  y  19.0  89.0  20.0  57.0  70.0    NaN
  z  98.0   1.0  26.0  65.0  36.0    3.0
c z  44.0  55.0  36.0  50.0  12.0   83.0

In [16]:
# join也可以将列和索引进行合并，需指明作为连接键的列
# 注意要让有连接键列的表来调用join方法
data2.join(data1,on=['key1','key2'])

,key1,key2,value,A,B,C,D,E
0,b,x,X,NaN,NaN,NaN,NaN,NaN
1,b,z,Z,98.0,1.0,26.0,65.0,36.0
2,a,x,Y,11.0,71.0,43.0,41.0,60.0
3,a,y,Z,78.0,88.0,73.0,89.0,34.0
4,c,z,X,44.0,55.0,36.0,50.0,12.0


In [17]:
data4=data3.copy()
data4.columns=['data4_value'] # join不能有重叠的列，因此要修改列名
data4

data4_value
b x           82
  z            3
a x           75
  y           39
c z           83

In [18]:
# 使用join按索引合并多个表，以列表形式传入要合并的表
data1.join([data3,data4])

A   B   C   D   E  value  data4_value
a x  11  71  43  41  60   75.0         75.0
  y  78  88  73  89  34   39.0         39.0
b y  19  89  20  57  70    NaN          NaN
  z  98   1  26  65  36    3.0          3.0
c z  44  55  36  50  12   83.0         83.0

## 3. 合并重叠数据